# 020-rede2018_converte_receitas

In [1]:
ano_eleicao = '2018'

In [2]:
dbschema = f'rede{ano_eleicao}'
dbschema_tse = f'tse{ano_eleicao}'
table_receitas = f'{dbschema}.receitas_{ano_eleicao}'
table_receitas_sdo = f'{dbschema}.receitas_sdo_{ano_eleicao}'
table_receitas_candidatos = f'{dbschema_tse}.receitas_candidatos_{ano_eleicao}'
table_receitas_candidatos_doador_originario = f'{dbschema_tse}.receitas_candidatos_doador_originario_{ano_eleicao}'
table_receitas_orgaos_partidarios = f'{dbschema_tse}.receitas_orgaos_partidarios_{ano_eleicao}'
table_receitas_orgaos_partidarios_doador_originario= f'{dbschema_tse}.receitas_orgaos_partidarios_doador_originario_{ano_eleicao}'
table_consulta_cand = f'{dbschema_tse}.consulta_cand_{ano_eleicao}'
table_candidaturas = f"{dbschema}.candidaturas_{ano_eleicao}"
table_municipios = f"tse{ano_eleicao}.municipios_{ano_eleicao}"
table_partidos = f'{dbschema_tse}.partidos_{ano_eleicao}'
table_orgaos_partidarios =  f"{dbschema}.orgaos_partidarios_{ano_eleicao}"
table_origem_receitas = f'tse.origem_receitas'
table_fonte_receitas = f'tse.fonte_receitas'

In [3]:
import os
import sys
sys.path.append('../')
import mod_tse as mtse
home = os.environ["HOME"]

In [4]:
mtse.execute_query(f'CREATE SCHEMA IF NOT EXISTS {dbschema};')

## CRIA TABELA RECEITAS

In [5]:
query_create_table_receitas = f"""
DROP TABLE IF EXISTS {table_receitas} CASCADE;
CREATE TABLE {table_receitas} (
    tabela_id                     varchar,

    geracao_data                   varchar,
    geracao_hora                   varchar,
    eleicao_ano                    varchar,
    eleicao_cd                     varchar,
    eleicao_ds                     varchar,
    eleicao_tipo_cd                varchar,
    eleicao_tipo_ds                varchar,
    eleicao_data                   varchar,
    eleicao_turno                  varchar,
    prestacao_conta_tp             varchar,
    prestacao_conta_data           varchar,
    prestador_contas_sq            varchar,

    --RECEITA
    receita_num_recibo             varchar,
    receita_num_doc                varchar,
    receita_fonte_cd               varchar,
    receita_fonte_ds               varchar,
    receita_origem_cd              varchar,
    receita_origem_sg              varchar,
    receita_origem_ds              varchar,
    receita_sq                     varchar,
    receita_natureza_cd            varchar,
    receita_natureza_ds            varchar,
    receita_especie_cd             varchar,
    receita_especie_ds             varchar,
    receita_data                   varchar,
    receita_descricao              varchar,
    receita_valor                  numeric(18,2),

    --RECEPTOR
    receptor_id                    varchar,
    receptor_sequencial            varchar,
    receptor_tipo_cd               varchar,
    receptor_subtipo_cd            varchar,
    receptor_tipo_ds               varchar,
    receptor_subtipo_ds            varchar,
    receptor_label                 varchar,
    
    receptor_genero_ds             varchar,
    receptor_esfera_partidaria_cd  varchar,
    receptor_esfera_partidaria_ds  varchar,
    receptor_uf                    varchar,
    receptor_ue                    varchar,
    receptor_ue_nome               varchar,
    receptor_cnpj                  varchar,
    receptor_cargo_cd              varchar,
    receptor_cargo_ds              varchar,
    receptor_candidato_seq         varchar,
    receptor_candidato_nr          varchar,
    receptor_candidato_nome        varchar,
    receptor_candidato_cpf         varchar,
    receptor_vice_candidato_cpf    varchar,
    receptor_partido_nr            varchar,
    receptor_partido_sg            varchar,
    receptor_partido_nome          varchar,

    --DOADOR
    doador_id                      varchar,
    doador_tipo_cd                 varchar,
    doador_subtipo_cd              varchar,
    doador_tipo_ds                 varchar,
    doador_subtipo_ds              varchar,
    doador_label                   varchar,
    doador_genero_ds               varchar,
    
    doador_cnae_cd                 varchar,
    doador_cnae_ds                 varchar,
    doador_cpf_cnpj                varchar,
    doador_nome                    varchar,
    doador_nome_rfb                varchar,
    doador_originario_tipo         varchar,
    doador_esfera_partidaria_cd    varchar,  
    doador_esfera_partidaria_ds    varchar,
    doador_uf                      varchar,
    doador_ue                      varchar,
    doador_ue_nome                 varchar,
    doador_candidato_sq            varchar,
    doador_candidato_nr            varchar,
    doador_candidato_nome          varchar,
    doador_candidato_cargo_cd      varchar,
    doador_candidato_cargo_ds      varchar,
    doador_partido_nr              varchar,
    doador_partido_sg              varchar,
    doador_partido_nome             varchar
);

CREATE INDEX ON {table_receitas} (tabela_id);
CREATE INDEX ON {table_receitas} (receita_origem_sg);
CREATE INDEX ON {table_receitas} (receita_origem_cd);
CREATE INDEX ON {table_receitas} (receita_origem_ds);

CREATE INDEX ON {table_receitas} (receptor_id);
CREATE INDEX ON {table_receitas} (receptor_tipo_cd);
CREATE INDEX ON {table_receitas} (receptor_subtipo_cd);
CREATE INDEX ON {table_receitas} (receptor_cnpj);
CREATE INDEX ON {table_receitas} (receptor_candidato_cpf);
CREATE INDEX ON {table_receitas} (receptor_uf); 
CREATE INDEX ON {table_receitas} (receptor_candidato_nr); 
CREATE INDEX ON {table_receitas} (receptor_candidato_seq);
CREATE INDEX ON {table_receitas} (receptor_sequencial);


CREATE INDEX ON {table_receitas} (doador_id);
CREATE INDEX ON {table_receitas} (doador_tipo_cd);
CREATE INDEX ON {table_receitas} (doador_subtipo_cd);
CREATE INDEX ON {table_receitas} (doador_candidato_sq);
CREATE INDEX ON {table_receitas} (doador_cpf_cnpj);
CREATE INDEX ON {table_receitas} (doador_nome_rfb);
CREATE INDEX ON {table_receitas} (doador_nome); 

CREATE VIEW {table_receitas_sdo} as
( select * from {table_receitas}
  where tabela_id in ('RC','ROP')
)
"""

mtse.execute_query(query_create_table_receitas)
print("Tebela Receitas Criada.")

Tebela Receitas Criada.


## INCLUI RECEITAS DE CANDIDATOS

In [9]:
query_insert_receitas_candidatos = f"""
insert into {table_receitas}
select  
    'RC'                         as       tabela_id,
    dt_geracao                   as       geracao_data,
    hh_geracao                   as       geracao_hora,
    ano_eleicao                  as       eleicao_ano,
    cd_tipo_eleicao              as       eleicao_cd,
    nm_tipo_eleicao              as       eleicao_ds,
    cd_eleicao                   as       eleicao_tipo_cd,
    ds_eleicao                   as       eleicao_tipo_ds,
    dt_eleicao                   as       eleicao_data,
    st_turno                     as       eleicao_turno,
    tp_prestacao_contas          as       prestacao_conta_tp,
    dt_prestacao_contas          as       prestacao_conta_data,
    sq_prestador_contas          as       prestador_contas_sq,
                --RECEITA  
    nr_recibo_doacao             as       receita_num_recibo,
    nr_documento_doacao          as       receita_num_doc,
    cd_fonte_receita             as       receita_fonte_cd,
    ds_fonte_receita             as       receita_fonte_ds,
    cd_origem_receita            as       receita_origem_cd,
    '#NE'                        as       receita_origem_sg,
    ds_origem_receita            as       receita_origem_ds,
    sq_receita                   as       receita_sq,
    cd_natureza_receita          as       receita_natureza_cd,
    ds_natureza_receita          as       receita_natureza_ds,
    cd_especie_receita           as       receita_especie_cd,
    ds_especie_receita           as       receita_especie_ds,
    dt_receita                   as       receita_data,
    ds_receita                   as       receita_descricao,
    vr_receita::numeric(18,2)    as       receita_valor,
                 --RECEPTOR    
    get_candidato_id(sg_uf,sq_candidato)     as       receptor_id,
    sq_prestador_contas                      as       receptor_sequencial,
    'CA'                                     as       receptor_tipo_cd,
    'CA'                                     as       receptor_subtipo_cd,
    'Candidato'                              as       receptor_tipo_ds,
    'Candidato'                              as       receptor_subtipo_ds,
    get_candidato_label(nm_candidato,ds_cargo,sg_uf,sg_partido) as receptor_label, 
    '#NE'                        as       receptor_genero_ds,   
    '#NE'                        as       receptor_esfera_partidaria_cd,
    '#NE'                        as       receptor_esfera_partidaria_ds,
    sg_uf                        as       receptor_uf,
    sg_ue                        as       receptor_ue,
    nm_ue                        as       receptor_ue_nome,
    nr_cnpj_prestador_conta      as       receptor_cnpj,
    cd_cargo                     as       receptor_cargo_cd,
    ds_cargo                     as       receptor_cargo_ds,
    sq_candidato                 as       receptor_candidato_seq,
    nr_candidato                 as       receptor_candidato_nr,
    nm_candidato                 as       receptor_candidato_nome,
    nr_cpf_candidato             as       receptor_candidato_cpf,
    nr_cpf_vice_candidato        as       receptor_vice_candidato_cpf,
    nr_partido                   as       receptor_partido_nr,
    sg_partido                   as       receptor_partido_sg,
    nm_partido                   as       receptor_partido_nome,
    --DOADOR   
    get_doador_id(nr_cpf_cnpj_doador)           as       doador_id,   
    get_doador_tipo(nr_cpf_cnpj_doador)         as       doador_tipo_cd,
    get_doador_tipo(nr_cpf_cnpj_doador)         as       doador_subtipo_cd, 
    get_doador_tipo_ds(nr_cpf_cnpj_doador)      as       doador_tipo_ds,
    get_doador_tipo_ds(nr_cpf_cnpj_doador)      as       doador_subtipo_ds,
    get_doador_label(nm_doador_rfb, nm_doador)  as       doador_label,    
    '#NE'                        as       doador_genero_ds,  
    cd_cnae_doador               as       doador_cnae_cd,
    ds_cnae_doador               as       doador_cnae_ds,
    nr_cpf_cnpj_doador           as       doador_cpf_cnpj,
    nm_doador                    as       doador_nome,
    nm_doador_rfb                as       doador_nome_rfb,
    '#NE'                        as       doador_originario_tipo,
    cd_esfera_partidaria_doador  as       doador_esfera_partidaria_cd,  
    ds_esfera_partidaria_doador  as       doador_esfera_partidaria_ds,
    sg_uf_doador                 as       doador_uf,
    cd_municipio_doador          as       doador_ue,
    nm_municipio_doador          as       doador_ue_nome,
    sq_candidato_doador          as       doador_candidato_sq,
    nr_candidato_doador          as       doador_candidato_nr,
    '#NE'                        as       doador_candidato_nome,
    cd_cargo_candidato_doador    as       doador_candidato_cargo_cd,
    ds_cargo_candidato_doador    as       doador_candidato_cargo_ds,
    nr_partido_doador            as       doador_partido_nr,
    sg_partido_doador            as       doador_partido_sg,
    nm_partido_doador            as       doador_partido_nome
from {table_receitas_candidatos}
;
"""

mtse.execute_query(query_insert_receitas_candidatos)

## INCLUI RECEITAS DE CANDIDATOS DOADOR ORIGINÁRIO

In [13]:
query_insert_receitas_candidatos_doador_originario = f"""
insert into {table_receitas}
select   
    'RCDO'                               as           tabela_id,
        
    rcdo.dt_geracao                      as           geracao_data,
    rcdo.hh_geracao                      as           geracao_hora,
    rcdo.ano_eleicao                     as           eleicao_ano,
    rcdo.cd_tipo_eleicao                 as           eleicao_cd,
    rcdo.nm_tipo_eleicao                 as           eleicao_ds,
    rcdo.cd_eleicao                      as           eleicao_tipo_cd,
    rcdo.ds_eleicao                      as           eleicao_tipo_ds,
    rcdo.dt_eleicao                      as           eleicao_data,
    rcdo.st_turno                        as           eleicao_turno,
    rcdo.tp_prestacao_contas             as           prestacao_conta_tp,
    rcdo.dt_prestacao_contas             as           prestacao_conta_data,
    rcdo.sq_prestador_contas             as           prestador_contas_sq,
        
                     --RECEITA           
    '#NE'                                as           receita_num_recibo,
    '#NE'                                as           receita_num_doc,
    '#NE'                                as           receita_fonte_cd,
    '#NE'                                as           receita_fonte_ds,
    '#NE'                                as           receita_origem_cd,
    '#NE'                                as           receita_origem_sg,
    '#NE'                                as           receita_origem_ds,
    rcdo.sq_receita                      as           receita_sq,
    '#NE'                                as           receita_natureza_cd,
    '#NE'                                as           receita_natureza_ds,
    '#NE'                                as           receita_especie_cd,
    '#NE'                                as           receita_especie_ds,
    rcdo.dt_receita                      as           receita_data,
    rcdo.ds_receita                      as           receita_descricao,
    rcdo.vr_receita::numeric(18,2)       as           receita_valor,
                --RECEPTOR            
    '#NE'                                as           receptor_id,
    rcdo.sq_prestador_contas             as           receptor_sequencial,
    'CA'                                 as           receptor_tipo_cd,
    'DO'                                 as           receptor_subtipo_cd,
    'Candidato'                          as           receptor_tipo_ds,
    'Candidato Doador Originário'        as           receptor_subtipo_ds,
    'Candidato - ' || rcdo.sg_uf          as           receptor_label,
    '#NE'                                as           receptor_genero_ds,   
    '#NE'                                as           receptor_esfera_partidaria_cd,
    '#NE'                                as           receptor_esfera_partidaria_ds,
    rcdo.sg_uf                           as           receptor_uf,
    '#NE'                                as           receptor_ue,
    '#NE'                                as           receptor_ue_nome,
    '#NE'                                as           receptor_cnpj,
    '#NE'                                as           receptor_cargo_cd,
    '#NE'                                as           receptor_cargo_ds,
    '#NE'                                as           receptor_candidato_seq,
    '#NE'                                as           receptor_candidato_nr,
    '#NE'                                as           receptor_candidato_nome,
    '#NE'                                as           receptor_candidato_cpf,
    '#NE'                                as           receptor_vice_candidato_cpf,
    '#NE'                                as           receptor_partido_nr,
    '#NE'                                as           receptor_partido_sg,
    '#NE'                                as           receptor_partido_nome,
                --DOADOR            
    get_doador_id(rcdo.nr_cpf_cnpj_doador_originario )          as       doador_id,   
    get_doador_tipo(rcdo.nr_cpf_cnpj_doador_originario)         as       doador_tipo_cd,
    get_doador_tipo(rcdo.nr_cpf_cnpj_doador_originario)         as       doador_subtipo_cd,
    get_doador_tipo_ds(rcdo.nr_cpf_cnpj_doador_originario)      as       doador_tipo_ds,
    get_doador_tipo_ds(rcdo.nr_cpf_cnpj_doador_originario)      as       doador_subtipo_ds,      
    get_doador_label(rcdo.nm_doador_originario_rfb, rcdo.nm_doador_originario) as doador_label,
    '#NE'                                as           doador_genero_ds,
    rcdo.cd_cnae_doador_originario       as           doador_cnae_cd,
    rcdo.ds_cnae_doador_originario       as           doador_cnae_ds,
    rcdo.nr_cpf_cnpj_doador_originario   as           doador_cpf_cnpj,
    rcdo.nm_doador_originario            as           doador_nome,
    rcdo.nm_doador_originario_rfb        as           doador_nome_rfb,
    rcdo.tp_doador_originario            as           doador_originario_tipo,
    '#NE'                                as           doador_esfera_partidaria_cd,  
    '#NE'                                as           doador_esfera_partidaria_ds,
    '#NE'                                as           doador_uf,
    '#NE'                                as           doador_ue,
    '#NE'                                as           doador_ue_nome,
    '#NE'                                as           doador_candidato_sq,
    '#NE'                                as           doador_candidato_nr,
    '#NE'                                as           doador_candidato_nome,
    '#NE'                                as           doador_candidato_cargo_cd,
    '#NE'                                as           doador_candidato_cargo_ds,
    '#NE'                                as           doador_partido_nr,
    '#NE'                                as           doador_partido_sg,
    '#NE'                                as           doador_partido_nome
from {table_receitas_candidatos_doador_originario} as  rcdo
;
"""

mtse.execute_query(query_insert_receitas_candidatos_doador_originario)


## CODIFICA GENERO DO CANDIDATO

In [14]:
query_update_genero = f"""

CREATE INDEX ON {table_consulta_cand} (sq_candidato);
CREATE INDEX ON {table_receitas} (receptor_candidato_seq);
CREATE INDEX ON {table_receitas} (receptor_sequencial);

with candidaturas as
(
select ds_genero, sq_candidato from {table_consulta_cand}
group by ds_genero, sq_candidato
)
update {table_receitas} as r
    set receptor_genero_ds = c.ds_genero
from candidaturas as c
    where c.sq_candidato = r.receptor_candidato_seq
    or 
    c.sq_candidato = r.receptor_sequencial
;
"""

mtse.execute_query(query_update_genero)


## INCLUI RECEITAS ORGÃOS PARTIDÁRIOS

In [15]:
query_insert_receitas_orgaos_partidarios = f"""
insert into {table_receitas}
select 
   'ROP'                            as           tabela_id,       
    dt_geracao                      as           geracao_data,
    hh_geracao                      as           geracao_hora,
    ano_eleicao                     as           eleicao_ano,
    cd_tipo_eleicao                 as           eleicao_cd,
    nm_tipo_eleicao                 as           eleicao_ds,
    '#NE'                           as           eleicao_tipo_cd,
    '#NE'                           as           eleicao_tipo_ds,
    '#NE'                           as           eleicao_data,
    '#NE'                           as           eleicao_turno,
    tp_prestacao_contas             as           prestacao_conta_tp,
    dt_prestacao_contas             as           prestacao_conta_data,
    sq_prestador_contas             as           prestador_contas_sq,   
                 --RECEITA           
    nr_recibo_doacao                as           receita_num_recibo,
    nr_documento_doacao             as           receita_num_doc,
    cd_fonte_receita                as           receita_fonte_cd,
    ds_fonte_receita                as           receita_fonte_ds,
    cd_origem_receita               as           receita_origem_cd,
    '#NE'                           as           receita_origem_sg,
    ds_origem_receita               as           receita_origem_ds,
    sq_receita                      as           receita_sq,
    cd_natureza_receita             as           receita_natureza_cd,
    ds_natureza_receita             as           receita_natureza_ds,
    cd_especie_receita              as           receita_especie_cd,
    ds_especie_receita              as           receita_especie_ds,
    dt_receita                      as           receita_data,
    ds_receita                      as           receita_descricao,
    vr_receita::numeric(18,2)       as           receita_valor,    
                 --RECEPTOR         
    public.get_orgao_partidario_id('OP',nr_cnpj_prestador_conta) as receptor_id,
    sq_prestador_contas             as           receptor_sequencial,
    'OP'                            as           receptor_tipo_cd,
    cd_esfera_partidaria            as           receptor_subtipo_cd,
    'Orgão Partidário'              as           receptor_tipo_ds,
    ds_esfera_partidaria            as           receptor_subtipo_ds,
    public.get_orgao_partidario_label(sg_partido,sg_uf,ds_esfera_partidaria) as receptor_label,              
    '#NE'                           as           receptor_genero_ds,
    cd_esfera_partidaria            as           receptor_esfera_partidaria_cd,
    ds_esfera_partidaria            as           receptor_esfera_partidaria_ds,
    sg_uf                           as           receptor_uf,
    cd_municipio                    as           receptor_ue,
    nm_municipio                    as           receptor_ue_nome,
    nr_cnpj_prestador_conta         as           receptor_cnpj,
    '#NE'                           as           receptor_cargo_cd,
    '#NE'                           as           receptor_cargo_ds,
    '#NE'                           as           receptor_candidato_seq,
    '#NE'                           as           receptor_candidato_nr,
    '#NE'                           as           receptor_candidato_nome,
    '#NE'                           as           receptor_candidato_cpf,
    '#NE'                           as           receptor_vice_candidato_cpf,
    nr_partido                      as           receptor_partido_nr,
    sg_partido                      as           receptor_partido_sg,
    nm_partido                      as           receptor_partido_nome,    
                 --DOADOR            
    get_doador_id( nr_cpf_cnpj_doador)          as  doador_id,
    get_doador_tipo(nr_cpf_cnpj_doador)         as  doador_tipo_cd,
    get_doador_tipo(nr_cpf_cnpj_doador)         as  doador_subtipo_cd,
    get_doador_tipo_ds(nr_cpf_cnpj_doador)      as  doador_tipo_ds,
    get_doador_tipo_ds(nr_cpf_cnpj_doador)      as  doador_subtipo_ds,
    get_doador_label(nm_doador_rfb, nm_doador)  as  doador_label,
    '#NE'                           as           doador_genero_ds,
    cd_cnae_doador                  as           doador_cnae_cd,
    ds_cnae_doador                  as           doador_cnae_ds,
    nr_cpf_cnpj_doador              as           doador_cpf_cnpj,
    nm_doador                       as           doador_nome,
    nm_doador_rfb                   as           doador_nome_rfb,
    '#NE'                           as           doador_originario_tipo,
    cd_esfera_partidaria_doador     as           doador_esfera_partidaria_cd,  
    ds_esfera_partidaria_doador     as           doador_esfera_partidaria_ds,
    sg_uf_doador                    as           doador_uf,
    cd_municipio_doador             as           doador_ue,
    nm_municipio_doador             as           doador_ue_nome,
    sq_candidato_doador             as           doador_candidato_sq,
    nr_candidato_doador             as           doador_candidato_nr,
    '#NE'                           as           doador_candidato_nome,
    cd_cargo_candidato_doador       as           doador_candidato_cargo_cd,
    ds_cargo_candidato_doador       as           doador_candidato_cargo_ds,
    nr_partido_doador               as           doador_partido_nr,
    sg_partido_doador               as           doador_partido_sg,
    nm_partido_doador               as           doador_partido_nome
from {table_receitas_orgaos_partidarios}
;
"""

mtse.execute_query(query_insert_receitas_orgaos_partidarios)

## INCLUI RECEITAS ORGÃOS PARTIDÁRIOS DOADOR ORIGINÁRIO

In [16]:
query_insert_receitas_orgaos_partidarios_doador_originario = f"""
insert into {table_receitas}
select
    'ROPDO'                    as       tabela_id,
    rpdo.dt_geracao            as       geracao_data,
    rpdo.hh_geracao            as       geracao_hora,
    rpdo.ano_eleicao           as       eleicao_ano,
    rpdo.cd_tipo_eleicao       as       eleicao_cd,
    rpdo.nm_tipo_eleicao       as       eleicao_ds,
    '#NE'                      as       eleicao_tipo_cd,
    '#NE'                      as       eleicao_tipo_ds,
    '#NE'                      as       eleicao_data,
    '#NE'                      as       eleicao_turno,
    rpdo.tp_prestacao_contas   as       prestacao_conta_tp,
    rpdo.dt_prestacao_contas   as       prestacao_conta_data,
    rpdo.sq_prestador_contas   as       prestador_contas_sq,
                     --RECEITA   
    '#NE'                      as       receita_num_recibo,
    '#NE'                      as       receita_num_doc,
    '#NE'                      as       receita_fonte_cd,
    '#NE'                      as       receita_fonte_ds,
    '#NE'                      as       receita_origem_cd,
    '#NE'                      as       receita_origem_sg,
    '#NE'                      as       receita_origem_ds,
    rpdo.sq_receita            as       receita_sq,
    '#NE'                      as       receita_natureza_cd,
    '#NE'                      as       receita_natureza_ds,
    '#NE'                      as       receita_especie_cd,
    '#NE'                      as       receita_especie_ds,
    rpdo.dt_receita            as       receita_data,
    rpdo.ds_receita            as       receita_descricao,
    rpdo.vr_receita::numeric(18,2)   as    receita_valor,
                --RECEPTOR    
    '#NE'                      as       receptor_id,
    rpdo.sq_prestador_contas   as       receptor_sequencial,
    'OP'                       as       receptor_tipo_cd,
    'DO'                       as       receptor_subtipo_cd,
    'Orgão Partidário'         as       receptor_tipo_ds,
     '#NE'                     as       receptor_subtipo_ds,
    'Orgão Partidário - '|| rpdo.sg_uf  as       receptor_label,    
    '#NE'                      as       receptor_genero_ds,  
    '#NE'                      as       receptor_esfera_partidaria_cd,
    '#NE'                      as       receptor_esfera_partidaria_ds,
    rpdo.sg_uf                 as       receptor_uf,
    '#NE'                      as       receptor_ue,
    '#NE'                      as       receptor_ue_nome,
    '#NE'                      as       receptor_cnpj,
    '#NE'                      as       receptor_cargo_cd,
    '#NE'                      as       receptor_cargo_ds,
    '#NE'                      as       receptor_candidato_seq,
    '#NE'                      as       receptor_candidato_nr,
    '#NE'                      as       receptor_candidato_nome,
    '#NE'                      as       receptor_candidato_cpf,
    '#NE'                      as       receptor_vice_candidato_cpf,
    '#NE'                      as       receptor_partido_nr,
    '#NE'                      as       receptor_partido_sg,
    '#NE'                      as       receptor_partido_nome,
                --DOADOR   
    get_doador_id( nr_cpf_cnpj_doador_originario)     as  doador_id,
    get_doador_tipo(nr_cpf_cnpj_doador_originario )   as  doador_tipo_cd,
    get_doador_tipo(nr_cpf_cnpj_doador_originario )   as  doador_subtipo_cd,
    get_doador_tipo_ds(nr_cpf_cnpj_doador_originario) as  doador_tipo_ds,
    get_doador_tipo_ds(nr_cpf_cnpj_doador_originario) as  doador_subtipo_ds,
    get_doador_label(rpdo.nm_doador_originario_rfb, rpdo.nm_doador_originario) as  doador_label,
    '#NE'                                as  doador_genero_ds,  
    rpdo.cd_cnae_doador_originario       as       doador_cnae_cd,
    rpdo.ds_cnae_doador_originario       as       doador_cnae_ds,
    rpdo.nr_cpf_cnpj_doador_originario   as       doador_cpf_cnpj,
    rpdo.nm_doador_originario            as       doador_nome,
    rpdo.nm_doador_originario_rfb        as       doador_nome_rfb,
    rpdo.tp_doador_originario            as       doador_originario_tipo,
    '#NE'                      as       doador_esfera_partidaria_cd,  
    '#NE'                      as       doador_esfera_partidaria_ds,
    '#NE'                      as       doador_uf,
    '#NE'                      as       doador_ue,
    '#NE'                      as       doador_ue_nome,
    '#NE'                      as       doador_candidato_sq,
    '#NE'                      as       doador_candidato_nr,
    '#NE'                      as       doador_candidato_nome,
    '#NE'                      as       doador_candidato_cargo_cd,
    '#NE'                      as       doador_candidato_cargo_ds,
    '#NE'                      as       doador_partido_nr,
    '#NE'                      as       doador_partido_sg,
    '#NE'                      as       doador_partido_nome
from {table_receitas_orgaos_partidarios_doador_originario} as rpdo
;
"""

mtse.execute_query(query_insert_receitas_orgaos_partidarios_doador_originario)

## INCLUI DADOS DE RECEPTOR EM DADOR ORIGINÁRIO DE ORGÃOS PARTIDÁRIOS E CANDIDATOS

In [17]:
query_atualiza_receptor_em_doador_originario = f"""
with receitas as 
(
select 
    tabela_id,
    receptor_id,
    receptor_sequencial,
    receptor_tipo_cd,
    receptor_subtipo_cd,    
    receptor_tipo_ds,
    receptor_subtipo_ds,
    receptor_label,
    receptor_genero_ds,
    receptor_esfera_partidaria_cd,
    receptor_esfera_partidaria_ds,
    receptor_uf,
    receptor_ue,
    receptor_ue_nome,
    receptor_cnpj,
    receptor_cargo_cd,
    receptor_cargo_ds,
    receptor_candidato_seq,
    receptor_candidato_nr,
    receptor_candidato_nome,
    receptor_candidato_cpf,
    receptor_vice_candidato_cpf,
    receptor_partido_nr,
    receptor_partido_sg,
    receptor_partido_nome
from {table_receitas}
where 
    tabela_id in ('ROP','RC')
)
update {table_receitas} as rdo
    set 
    receptor_id                    =    r.receptor_id,
    receptor_sequencial            =    r.receptor_sequencial,
    receptor_tipo_cd               =    r.receptor_tipo_cd,
    receptor_subtipo_cd            =    r.receptor_subtipo_cd,    
    receptor_tipo_ds               =    r.receptor_tipo_ds,
    receptor_subtipo_ds            =    r.receptor_subtipo_ds,
    receptor_label                 =    r.receptor_label,
    receptor_genero_ds             =    r.receptor_genero_ds,
    receptor_esfera_partidaria_cd  =    r.receptor_esfera_partidaria_cd,
    receptor_esfera_partidaria_ds  =    r.receptor_esfera_partidaria_ds,
--  receptor_uf                    =    r.receptor_uf,
    receptor_ue                    =    r.receptor_ue,
    receptor_ue_nome               =    r.receptor_ue_nome,
    receptor_cnpj                  =    r.receptor_cnpj,
    receptor_cargo_cd              =    r.receptor_cargo_cd,
    receptor_cargo_ds              =    r.receptor_cargo_ds,
    receptor_candidato_seq         =    r.receptor_candidato_seq,
    receptor_candidato_nr          =    r.receptor_candidato_nr,
    receptor_candidato_nome        =    r.receptor_candidato_nome,
    receptor_candidato_cpf         =    r.receptor_candidato_cpf,
    receptor_vice_candidato_cpf    =    r.receptor_vice_candidato_cpf,
    receptor_partido_nr            =    r.receptor_partido_nr,
    receptor_partido_sg            =    r.receptor_partido_sg,
    receptor_partido_nome          =    r.receptor_partido_nome
from receitas as r
  where (r.receptor_sequencial = rdo.receptor_sequencial and rdo.tabela_id = 'ROPDO')
     or (r.receptor_sequencial = rdo.receptor_sequencial and rdo.tabela_id = 'RCDO')
;
"""

mtse.execute_query(query_atualiza_receptor_em_doador_originario)